In [ ]:
import sagemaker

# Preference
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

# Make model
pytorch_model = PyTorchModel(model_data="s3://sagemaker-studio-fltv8olasxt/eccv_final_model.tar.gz",
                             role=role,
                             framework_version="1.8.1",
                             py_version="py36",
                             source_dir="src",
                             entry_point="sm_entrypoint.py")

In [ ]:
# Deploy model
deploy_params = {
    "instance_type": "ml.t2.medium",
    "initial_instance_count": 1
}
predictor = pytorch_model.deploy(**deploy_params)

In [ ]:
# Test sagemaker endpoint
import json
import numpy as np
import boto3
runtime = boto3.Session().client(service_name='sagemaker-runtime')

# Load the image bytes
img = open('IncidentsDataset/example_images/fire.jpg', 'rb').read()

# Call your model for predicting which object appears in this image.
response = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    ContentType='image/jpeg',
    Body=bytearray(img)
)
response['Body'].read().decode('utf-8')